In [1]:
import pickle, os
from itertools import islice
import numpy as np, scipy as sp, pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcdefaults()
if hasattr(__builtins__, '__IPYTHON__'):
    get_ipython().run_line_magic('matplotlib', 'inline')
    get_ipython().run_line_magic('config', 'InlineBackend.figure_format = "jpg"')
    # https://stackoverflow.com/questions/70766518/how-to-change-ipython-error-highlighting-color
    from IPython.core import ultratb
    print(ultratb.VerboseTB._tb_highlight)
    ultratb.VerboseTB._tb_highlight = "bg:#006400"

bg:ansiyellow


In [2]:
from math import tan, sin, cos, radians
def latlon_2_twd97(lat, lon):
    # Modify from https://blog.ez2learn.com/2009/08/15/lat-lon-to-twd97/
    a = 6378137.0
    b = 6356752.314245
    long0 = radians(121)
    k0 = 0.9999
    dx = 250000
    
    lat, lon = radians(lat), radians(lon)

    e = (1-b**2/a**2)**0.5
    e2 = e**2/(1-e**2)
    n = (a-b)/(a+b)
    nu = a/(1-(e**2)*(sin(lat)**2))**0.5
    p = lon-long0

    A = a*(1 - n + (5/4.0)*(n**2 - n**3) + (81/64.0)*(n**4  - n**5))
    B = (3*a*n/2.0)*(1 - n + (7/8.0)*(n**2 - n**3) + (55/64.0)*(n**4 - n**5))
    C = (15*a*(n**2)/16.0)*(1 - n + (3/4.0)*(n**2 - n**3))
    D = (35*a*(n**3)/48.0)*(1 - n + (11/16.0)*(n**2 - n**3))
    E = (315*a*(n**4)/51.0)*(1 - n)

    S = A*lat - B*sin(2*lat) + C*sin(4*lat) - D*sin(6*lat) + E*sin(8*lat)

    K1 = S*k0
    K2 = k0*nu*sin(2*lat)/4.0
    K3 = (k0*nu*sin(lat)*(cos(lat)**3)/24.0) * \
        (5 - tan(lat)**2 + 9*e2*(cos(lat)**2) + 4*(e2**2)*(cos(lat)**4))

    y = K1 + K2*(p**2) + K3*(p**4)

    K4 = k0*nu*cos(lat)
    K5 = (k0*nu*(cos(lat)**3)/6.0) * \
        (1 - tan(lat)**2 + e2*(cos(lat)**2))

    x = K4*p + K5*(p**3) + dx
    return x, y

x20, y20 = latlon_2_twd97(23.9738833, 120.9820167) 
x20, y20 # 248170.787 2652129.936 https://www.sunriver.com.tw/grid_tm2.htm

(248169.98121661693, 2652130.8966781525)

In [3]:
x20_min, x20_max, y20_min, y20_max = 239000, 351800, 2744000, 2801720
x_min, x_max, y_min, y_max = 0, (x20_max - x20_min) // 20, 0, (y20_max - y20_min) // 20

In [4]:
def x20_2_x(x20):
    return (x20 - x20_min) // 20
def y20_2_y(y20):
    return (y20 - y20_min) // 20